<a href="https://colab.research.google.com/github/hayden-huynh/CSE-5334-TermProject/blob/master/CSE_5334_TermProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSE 5334 - Data Mining: Term Project

## About The Project
- Dataset: [Top Reddit Posts and Comments](https://www.kaggle.com/datasets/tushar5harma/topredditcomments?select=Top_Posts.csv)
- Goal: Given a comment of a post, classify which subreddit (Machine Learning, Artificial Intelligence, or Data Science) that it belongs to
- Solution: Naive Bayes Classifier for Text Classification

## Project Steps
1. Download the Top Reddit Posts and Comments [dataset](https://www.kaggle.com/datasets/tushar5harma/topredditcomments?select=Top_Posts.csv) from Kaggle
2. Based on the original dataset, formulate a dataframe consisting of a column of **comment text**, and a column of subreddit class
3. Perform text pre-processing
  - Lower-casing
  - Punctuation removal
  - Tokenization and duplicate word removal
4. Split the data into ***train*** (70%), ***dev*** (20%), and ***test*** (10%) subsets
5. Train the Naive Bayes Classifier
6. Experiment for optimization with **dev** dataset
  - Smoothing
  - Stopword removal
  - Stemming / Lemmatization
7. Conclude final accuracy with **test** dataset

# Download the Dataset

In [119]:
# Download the Top Reddit Posts & Reviews dataset from Kaggle
# Reference 1 (Ref 1): https://www.analyticsvidhya.com/blog/2021/06/how-to-load-kaggle-datasets-directly-into-google-colab/

# Ref 1 starts =====
! pip install kaggle
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download tushar5harma/topredditcomments
! unzip topredditcomments.zip
# ===== Ref 1 ends

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘/root/.kaggle’: File exists
topredditcomments.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  topredditcomments.zip
replace Top_Posts.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Top_Posts.csv           
replace Top_Posts_Comments.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Top_Posts_Comments.csv  


# Formulate Comment-Subreddit Dataframe

In [122]:
import pandas as pd
import numpy as np

top_posts = pd.read_csv("/content/Top_Posts.csv")
top_posts_comments = pd.read_csv("/content/Top_Posts_Comments.csv")

posts_id_class = top_posts[["post_id", "subreddit"]]

# Three classes: "MachineLearning", "datascience", "artificial"
comments = pd.merge(top_posts_comments, posts_id_class, on="post_id")

comments = comments[comments['comment'].notna()]

comments.sample(5)

,post_id,comment,subreddit
205581,ttfo0c,Or you are working in predictive maintenance,datascience
44187,hkiyir,"\> They cite to the SILU paper, which was publ...",MachineLearning
30430,8rdpwy,"Jesus, where to begin. It's a performance metr...",MachineLearning
222650,dt628c,It’s how all “dangerous” ML techniques will go...,artificial
230,kuc6tz,Agree - it was 6 years later until MNIST was e...,MachineLearning


# Perform text pre-processing

In [168]:
# Reference 2 (Ref 2): https://www.analyticsvidhya.com/blog/2021/06/text-preprocessing-in-nlp-with-python-codes/
import string
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk_stopwords = stopwords.words("english")

# ---------- Lower-casing ----------
# Ref 2 starts =====
comments["comment"] = comments["comment"].apply(lambda c : c.lower())
# ===== Ref 2 ends

# ---------- Tokenization and Duplicate removal ----------
# Ref 2 starts =====
def tokenize(text):
  tokens = re.split("\W+", text)
# ===== Ref 2 ends
  tokens = list(filter(None, tokens))
  return sorted(list(set(tokens)))

# Ref 2 starts =====
comments["tokens"] = comments["comment"].apply(lambda c: tokenize(c))
# ===== Ref 2 ends

# ---------- Stopword removal ----------
# Ref 2 starts =====
def remove_stopwords(words):
  output = [w for w in words if w not in nltk_stopwords]
  return output
# ===== Ref 2 ends

# Ref 2 starts =====
comments["tokens"] = comments["tokens"].apply(lambda words: remove_stopwords(words))
# ===== Ref 2 ends

# ---------- Punctuation removal ----------
# Ref 2 starts =====
def remove_punc_str(text):
  punc_free = "".join([char for char in text if char not in string.punctuation])
  return punc_free
# ===== Ref 2 ends

def remove_punc_arr(words):
  for i, w in enumerate(words):
    # Ref 2 starts =====
    punc_free = "".join([char for char in w if char not in string.punctuation])
    words[i] = punc_free
    # ===== Ref 2 ends
  return words

# Ref 2 starts =====
comments["tokens"] = comments["tokens"].apply(lambda c: remove_punc_arr(c))
# ===== Ref 2 ends

comments.sample(5)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,post_id,comment,subreddit,tokens
85216,jk9uy4,good luck! i hope you find something.,datascience,"[find, good, hope, luck, something]"
149733,3eu2rv,"this is brilliant! forget the ml, i need to up...",MachineLearning,"[brilliant, d3, forget, game, ml, need]"
140792,fq2k7w,i am in that camp. i do still use python but m...,datascience,"[analysis, better, camp, course, data, drawing..."
131137,tvp0nd,it goes both ways. candidates like me probably...,datascience,"[1, 2, actually, afford, application, better, ..."
52457,kagp2b,almost no states has cases at that level. you ...,datascience,"[almost, cases, idea, level, states, talking]"


# Split the Dataset

In [169]:
# Reference 3 (Ref 3): https://stackoverflow.com/questions/43777243/how-to-split-a-dataframe-in-pandas-in-predefined-percentages 

# Ref 3 starts =====
def split_by_fractions(df, fracs, random_state=0):
    remain = df.index.copy().to_frame()
    res = []
    for i in range(len(fracs)):
        fractions_sum = sum(fracs[i:])
        frac = fracs[i]/fractions_sum
        idxs = remain.sample(frac=frac, random_state=random_state).index
        remain=remain.drop(idxs)
        res.append(idxs)
    return [df.loc[idxs] for idxs in res]
# ===== Ref 3 ends

random_state = 1
train, dev, test = split_by_fractions(comments, [0.7, 0.2, 0.1], random_state)
print(train.shape, dev.shape, test.shape)

(156211, 4) (44632, 4) (22316, 4)


# Train the classifier

In [170]:
from decimal import Decimal

train_ml = train.loc[train["subreddit"] == "MachineLearning"]
train_ds = train.loc[train["subreddit"] == "datascience"]
train_ai = train.loc[train["subreddit"] == "artificial"]

# P(ml), P(ds), and P(ai) priors
p_ml = Decimal(len(train_ml) / len(train))
p_ds = Decimal(len(train_ds) / len(train))
p_ai = Decimal(len(train_ai) / len(train))

print(f'P(ml) = {p_ml}')
print(f'P(ds) = {p_ds}')
print(f'P(ai) = {p_ai}')

P(ml) = 0.427972421916510359363172710800427012145519256591796875
P(ds) = 0.4805679497602601824013390796608291566371917724609375
P(ai) = 0.09145962832322947211327601735320058651268482208251953125


In [171]:
# Count word occurences

occ_ml = {}
occ_ds = {}
occ_ai = {}

def count_occurrences(occ_dict, df):
  for words in df.loc[:,"tokens"]:
    for w in words:
      if w not in occ_dict.keys():
        occ_dict[w] = 1
      else:
        occ_dict[w] += 1

count_occurrences(occ_ml, train_ml)
count_occurrences(occ_ds, train_ds)
count_occurrences(occ_ai, train_ai)

In [172]:
# Calculate word probabilities given ml, ds, or ai

probs_ml = {}
probs_ds = {}
probs_ai = {}

def calc_word_likelihood(count, alpha, h):
  if h == "ml":
    return Decimal((count + alpha) / (len(train_ml) + alpha * 3))
  elif h == "ds":
    return Decimal((count + alpha) / (len(train_ds) + alpha * 3))
  elif h == "ai":
    return Decimal((count + alpha) / (len(train_ai) + alpha * 3))

def calc_prob(alpha=0):
  for word, count in occ_ml.items():
    probs_ml[word] = calc_word_likelihood(count, alpha, "ml")
  
  for word, count in occ_ds.items():
    probs_ds[word] = calc_word_likelihood(count, alpha, "ds")

  for word, count in occ_ai.items():
    probs_ai[word] = calc_word_likelihood(count, alpha, "ai")

alpha = 1

calc_prob(alpha)

# Validate with ***dev*** dataset

In [173]:
import csv
import os

# Function to classify a comment
def classify(comment_words, alpha):
  for w in comment_words:
    if w not in probs_ml.keys():
      probs_ml[w] = calc_word_likelihood(0, alpha, "ml")
    if w not in probs_ds.keys():
      probs_ds[w] = calc_word_likelihood(0, alpha, "ds")
    if w not in probs_ai.keys():
      probs_ai[w] = calc_word_likelihood(0, alpha, "ai")
  
  chance_ml = p_ml
  chance_ds = p_ds
  chance_ai = p_ai
  for w in comment_words:
    chance_ml = chance_ml * probs_ml[w]
    chance_ds = chance_ds * probs_ds[w]
    chance_ai = chance_ai * probs_ai[w]
  
  max_chance = max(chance_ml, chance_ds, chance_ai)

  if max_chance == chance_ml:
    return "MachineLearning"
  elif max_chance == chance_ds:
    return "datascience"
  else:
    return "artificial"
    

# Function to test entire dataset given
def test_accuracy(dataset, alpha, csv_writer=None):
  correct = 0
  
  for index, row in dataset.loc[:,["subreddit", "tokens"]].iterrows():
    result = classify(row["tokens"], alpha)
    if row["subreddit"] == result:
      correct += 1
  
  accuracy = round(correct / len(dataset) * 100, 4)

  if csv_writer != None:
    csv_writer.writerow([alpha, accuracy])
  
  print(f"Successfully classified {correct}/{len(dataset)} ({accuracy}%) correctly")

# ---------- Experiment with Smoothing ----------

# dev_smoothing = open(f"dev_smoothing.csv", "a", newline='')
# dev_smoothing_writer = csv.writer(dev_smoothing)
# if (os.path.getsize(f"/content/dev_smoothing.csv") == 0):
#   dev_smoothing_writer.writerow(["alpha", "accuracy"])

test_accuracy(dev, alpha)

# dev_smoothing.flush()

Successfully classified 31404/44632 (70.3621%) correctly


# Conclude final accuracy with ***test*** dataset

In [174]:
# test_smoothing = open(f'test_smoothing.csv', 'a', newline='')
# test_smoothing_writer = csv.writer(test_smoothing)
# if (os.path.getsize(f"/content/test_smoothing.csv") == 0):
#   dev_smoothing_writer.writerow(["alpha", "accuracy"])

test_accuracy(test, alpha)

# test_smoothing.flush()

Successfully classified 15663/22316 (70.1873%) correctly
